In [1]:
paths.dir("./")

{
  1 : ..
  2 : Code
  3 : nohup.out
  4 : .
  5 : DQN_Batch_CNN.ipynb
  6 : data
  7 : .gitignore
  8 : .git
  9 : Presentation
  10 : Untitled.ipynb
  11 : README.md
  12 : .ipynb_checkpoints
  13 : Paper
}


In [2]:
require 'os'
require 'nn'
require 'rnn'
require 'optim'
-- require 'cunn'
-- require 'cunnx'
-- require 'cutorch'
-- require 'parallel'

dl = require 'dataload'

In [3]:
dofile("Code/utils.lua")
dofile("Code/utilsNNbatch.lua")
dofile("Code/Utils/load_cnn.lua")

...Utils file loaded	


...Utils file loaded	


In [4]:
torch.setnumthreads(torch.getnumthreads())

In [5]:
outputpath = '/home/francisco/GitHub/DQN-Event-Summarization/data/training_ss/'
datafile = "cnn_data_ss.dat"

data = torch.load(outputpath .. datafile)

queries = data[1]
trueSummaries = data[2]
sentences= data[3]

In [6]:
vocab_size = 20001
learning_rate = 1e-5
embDim = 50
gamma = 0.
batch_size = 25
mem_multiplier = 1
cuts = 4
endexplorerate = 0.8
base_explore_rate = 0.1
nepochs = 100
epsilon = 1.
print_perf =false
adapt=false
adapt_lambda = 0.25
usecuda = false
seedval = 420

In [7]:
torch.manualSeed(seedval)
if usecuda then
    Tensor = torch.CudaTensor
    LongTensor = torch.CudaLongTensor   
    ByteTensor = torch.CudaByteTensor
    maskLayer = nn.MaskedSelect():cuda()
    print("...running on GPU")
else
    Tensor = torch.Tensor
    LongTensor = torch.LongTensor
    ByteTensor = torch.ByteTensor
    maskLayer = nn.MaskedSelect()
    print("...running on CPU")
end

SKIP = 1
SELECT = 2

k = sentences[1]:size(2)
n = queries:size(1)
q = queries:size(2)
n_s = #sentences

optimParams = { learningRate = learning_rate }
delta = cuts / nepochs
end_baserate = torch.round(nepochs * endexplorerate )


qTokens = {}
for i=1, n do
    qTokens[i] = Tokenize({trueSummaries[i]:totable()}, false)
end

-- Building the model
model = buildModel('bow', vocab_size, embDim, 'f1', adapt, usecuda)
params, gradParams = model:getParameters()

if adapt then 
    criterion = nn.ParallelCriterion():add(nn.MSECriterion()):add(nn.BCECriterion())
    criterion["weights"] = {1, adapt_lambda}
else 
    criterion = nn.MSECriterion()
end

...running on CPU	


Running bag-of-words model to learn f1	


In [8]:
print(string.format("Running model with %i queries and %i sentences", n, n_s)) 

In [9]:
mem_multiplier = 0.02
qValues = {}
qActions = {}
qPreds = {}
rewards = {}
lossfull = {}
rouguef1 = {}
rougue_scores = {}

totalPredsummary = LongTensor(n, n_s * k):fill(0)

memfull = false
curr_memsize = 0
memsize = torch.round(n * n_s * mem_multiplier)
queryMemory = Tensor(memsize, q):fill(0)
qActionMemory = Tensor(memsize, 2):fill(0)
predSummaryMemory = Tensor(memsize, n_s * k):fill(0)
sentenceMemory = Tensor(memsize, k):fill(0)
sentencetp1Memory  = Tensor(memsize, k):fill(0)
predSummarytp1Memory = Tensor(memsize, n_s * k):fill(0)
qPredsMemory = Tensor(memsize, 2):fill(0)
qValuesMemory = Tensor(memsize, 1):fill(0)
rewardMemory = Tensor(memsize, 1):fill(0)
totalPreds = Tensor(n, 2):fill(0)

Running model with 83566 queries and 122 sentences	


In [10]:
--- Initializing thingss
for i = 1, n_s do
    qPreds[i] = Tensor(n, 2):fill(0) 
    qValues[i] = Tensor(n, 1):fill(0)
    qActions[i] = Tensor(n, 2):fill(0)
    rewards[i] = Tensor(n, 1):fill(0)
    rougue_scores[i] = Tensor(n, 1):fill(0)
    if adapt then
        regPreds[i] = Tensor(n, 1):fill(0)
    end        
end 

In [11]:
if usecuda then
    criterion = criterion:cuda()
    model = model:cuda()
end

In [12]:
epoch = 1
i = 1

In [13]:
totalPreds:fill(0)
start_row = 1
end_row = batch_size
c = 1             
for start_row=1, n, batch_size do                 
    end_row = c * batch_size

    if end_row > n then 
        end_row = n
    end
    totalPreds[{{start_row, end_row}}]:copy(
        model:forward({
            queries[{{start_row, end_row}}], 
            sentences[i][{{start_row, end_row}}], 
            totalPredsummary[{{start_row, end_row}}]
        })
    )
    c = c + 1
end

In [14]:
if adapt then 
    qPreds[i]:copy(totalPreds[1])
    regPreds[i]:copy(totalPreds[2])
else
    qPreds[i]:copy(totalPreds)
end

In [15]:
if torch.uniform(0, 1) <= epsilon then
    -- randomly choosing actions
    xrand = torch.rand(qPreds[i]:size())
    qActions[i]:select(2, SELECT):copy(torch.ge(xrand:select(2, SELECT), xrand:select(2, SKIP)))
    qActions[i]:select(2, SKIP):copy(torch.ge(xrand:select(2, SKIP), xrand:select(2, SELECT)))
    qValues[i]:copy( maskLayer:forward({totalPreds, qActions[i]:byte()}) )
else 
    qMax, qindx = torch.max(qPreds[i], 2)  -- Pulling the best actions
    -- Here's the fast way to select the optimal action for each query
    qActions[i]:copy(
        qActions[i]:scatter(2, qindx, torch.ones(qPreds[i]:size())):clone()
    )
    qValues[i]:copy(
        qMax
    )
end

-- This is where we begin to store the data in our memory 
    -- notice that we store the reward after this part
start_row = curr_memsize + 1
if memsize < (start_row + n) then 
    start_row = memsize - n + 1
    end_row = start_row + n - 1
    curr_memsize = 0
    if (end_row + n) >= memsize then 
        memfull = true
    end 
else 
    end_row = start_row + n - 1
    curr_memsize = end_row
end

In [16]:
-- Update memory sequentially until it's full then restart updating it
queryMemory[{{start_row, end_row}}]:copy(queries)
sentenceMemory[{{start_row, end_row}}]:copy(sentences[i])
predSummaryMemory[{{start_row, end_row}}]:copy(totalPredsummary)

-- Now that we've stored our memory, we can build the summary to evaluate our action
predsummary = buildPredsummaryFast(qActions[i], sentences[i], SELECT)
totalPredsummary = buildTotalSummaryFast(predsummary, totalPredsummary, usecuda)

In [17]:
if i < n_s then
    sentencetp1Memory[{{start_row, end_row}}]:copy(sentences[i + 1])
    predSummarytp1Memory[{{start_row, end_row}}]:copy(totalPredsummary)
else 
    sentencetp1Memory[{{start_row, end_row}}]:copy(Tensor(sentences[i]:size()):fill(0) )
    predSummarytp1Memory[{{start_row, end_row}}]:copy(Tensor(totalPredsummary:size()):fill(0) )
end 

qActionMemory[{{start_row, end_row}}]:copy(qActions[i])
qPredsMemory[{{start_row, end_row}}]:copy(qPreds[i])
qValuesMemory[{{start_row, end_row}}]:copy(qValues[i])

In [19]:
out = 1
for j = 1, n do
    recall, prec, f1 = rougeScores( Tokenize({totalPredsummary[j]:totable()}),
                                    qTokens[j]
        )
    rougue_scores[i][j]:fill(f1)
    out = j
end

In [23]:
if i == n_s then 
    rouguef1[epoch] = rougue_scores[i]:mean()
end 

if i == 1 then
    -- Calculating change in rougue f1
    rewards[i]:copy(rougue_scores[i])
else 
    rewards[i]:copy(rougue_scores[i] - rougue_scores[i-1])
end
rewardMemory[{{start_row, end_row}}]:copy(rewards[i])

In [ ]:
for i=1, n_s do
    totalPreds:fill(0)
    start_row = 1
    end_row = batch_size
    c = 1             
    for start_row=1, n, batch_size do                 
        end_row = c * batch_size

        if end_row > n then 
            end_row = n
        end
        totalPreds[{{start_row, end_row}}]:copy(
            model:forward({
                queries[{{start_row, end_row}}], 
                sentences[i][{{start_row, end_row}}], 
                totalPredsummary[{{start_row, end_row}}]
            })
        )
        c = c + 1
    end

    if adapt then 
        qPreds[i]:copy(totalPreds[1])
        regPreds[i]:copy(totalPreds[2])
    else
        qPreds[i]:copy(totalPreds)
    end

    if torch.uniform(0, 1) <= epsilon then
        -- randomly choosing actions
        xrand = torch.rand(qPreds[i]:size())
        qActions[i]:select(2, SELECT):copy(torch.ge(xrand:select(2, SELECT), xrand:select(2, SKIP)))
        qActions[i]:select(2, SKIP):copy(torch.ge(xrand:select(2, SKIP), xrand:select(2, SELECT)))
        qValues[i]:copy( maskLayer:forward({totalPreds, qActions[i]:byte()}) )
    else 
        qMax, qindx = torch.max(qPreds[i], 2)  -- Pulling the best actions
        -- Here's the fast way to select the optimal action for each query
        qActions[i]:copy(
            qActions[i]:scatter(2, qindx, torch.ones(qPreds[i]:size())):clone()
        )
        qValues[i]:copy(
            qMax
        )
    end

    -- This is where we begin to store the data in our memory 
        -- notice that we store the reward after this part
    start_row = curr_memsize + 1
    if memsize < (start_row + n) then 
        start_row = memsize - n + 1
        end_row = start_row + n - 1
        curr_memsize = 0
        if (end_row + n) >= memsize then 
            memfull = true
        end 
    else 
        end_row = start_row + n - 1
        curr_memsize = end_row
    end

    -- Update memory sequentially until it's full then restart updating it
    queryMemory[{{start_row, end_row}}]:copy(queries)
    sentenceMemory[{{start_row, end_row}}]:copy(sentences[i])
    predSummaryMemory[{{start_row, end_row}}]:copy(totalPredsummary)

    -- Now that we've stored our memory, we can build the summary to evaluate our action
    predsummary = buildPredsummaryFast(qActions[i], sentences[i], SELECT)
    totalPredsummary = buildTotalSummaryFast(predsummary, totalPredsummary, usecuda)

    if i < n_s then
        sentencetp1Memory[{{start_row, end_row}}]:copy(sentences[i + 1])
        predSummarytp1Memory[{{start_row, end_row}}]:copy(totalPredsummary)
    else 
        sentencetp1Memory[{{start_row, end_row}}]:copy(Tensor(sentences[i]:size()):fill(0) )
        predSummarytp1Memory[{{start_row, end_row}}]:copy(Tensor(totalPredsummary:size()):fill(0) )
    end 

    qActionMemory[{{start_row, end_row}}]:copy(qActions[i])
    qPredsMemory[{{start_row, end_row}}]:copy(qPreds[i])
    qValuesMemory[{{start_row, end_row}}]:copy(qValues[i])

    if adapt then
        regMemory[{{start_row, end_row}}]:copy(regPreds[i])
    end

    for j = 1, n do
        recall, prec, f1 = rougeScores( Tokenize(totalPredsummary[j]:totable()),
                                        qTokens[j]
            )
        rougue_scores[i][j]:fill(f1)
    end

    if i == n_s then 
        rouguef1[epoch] = rougue_scores[i]:mean()
    end 

    if i == 1 then
        -- Calculating change in rougue f1
        rewards[i]:copy(rougue_scores[i])
    else 
        rewards[i]:copy(rougue_scores[i] - rougue_scores[i-1])
    end
    rewardMemory[{{start_row, end_row}}]:copy(rewards[i])
end


In [ ]:
print("training...")

nClock = os.clock()
for epoch=1, nepochs do
    --- Reset things at the start of each epoch
    for i=1, n_s do
        qPreds[i]:fill(0)
        qValues[i]:fill(0)
        qActions[i]:fill(0)
        rewards[i]:fill(0)
        rougue_scores[i]:fill(0)
        totalPredsummary:fill(0)
        if adapt then
            regMemory[i]:fill(0)
        end        
    end

    for i=1, n_s do
        totalPreds:fill(0)
        start_row = 1
        end_row = batch_size
        c = 1             
        for start_row=1, n, batch_size do                 
            end_row = c * batch_size

            if end_row > n then 
                end_row = n
            end
            totalPreds[{{start_row, end_row}}]:copy(
                model:forward({
                    queries[{{start_row, end_row}}], 
                    sentences[i][{{start_row, end_row}}], 
                    totalPredsummary[{{start_row, end_row}}]
                })
            )
            c = c + 1
        end

        if adapt then 
            qPreds[i]:copy(totalPreds[1])
            regPreds[i]:copy(totalPreds[2])
        else
            qPreds[i]:copy(totalPreds)
        end

        if torch.uniform(0, 1) <= epsilon then
            -- randomly choosing actions
            xrand = torch.rand(qPreds[i]:size())
            qActions[i]:select(2, SELECT):copy(torch.ge(xrand:select(2, SELECT), xrand:select(2, SKIP)))
            qActions[i]:select(2, SKIP):copy(torch.ge(xrand:select(2, SKIP), xrand:select(2, SELECT)))
            qValues[i]:copy( maskLayer:forward({totalPreds, qActions[i]:byte()}) )
        else 
            qMax, qindx = torch.max(qPreds[i], 2)  -- Pulling the best actions
            -- Here's the fast way to select the optimal action for each query
            qActions[i]:copy(
                qActions[i]:scatter(2, qindx, torch.ones(qPreds[i]:size())):clone()
            )
            qValues[i]:copy(
                qMax
            )
        end

        -- This is where we begin to store the data in our memory 
            -- notice that we store the reward after this part
        start_row = curr_memsize + 1
        if memsize < (start_row + n) then 
            start_row = memsize - n + 1
            end_row = start_row + n - 1
            curr_memsize = 0
            if (end_row + n) >= memsize then 
                memfull = true
            end 
        else 
            end_row = start_row + n - 1
            curr_memsize = end_row
        end

        -- Update memory sequentially until it's full then restart updating it
        queryMemory[{{start_row, end_row}}]:copy(queries)
        sentenceMemory[{{start_row, end_row}}]:copy(sentences[i])
        predSummaryMemory[{{start_row, end_row}}]:copy(totalPredsummary)

        -- Now that we've stored our memory, we can build the summary to evaluate our action
        predsummary = buildPredsummaryFast(qActions[i], sentences[i], SELECT)
        totalPredsummary = buildTotalSummaryFast(predsummary, totalPredsummary, usecuda)

        if i < n_s then
            sentencetp1Memory[{{start_row, end_row}}]:copy(sentences[i + 1])
            predSummarytp1Memory[{{start_row, end_row}}]:copy(totalPredsummary)
        else 
            sentencetp1Memory[{{start_row, end_row}}]:copy(Tensor(sentences[i]:size()):fill(0) )
            predSummarytp1Memory[{{start_row, end_row}}]:copy(Tensor(totalPredsummary:size()):fill(0) )
        end 

        qActionMemory[{{start_row, end_row}}]:copy(qActions[i])
        qPredsMemory[{{start_row, end_row}}]:copy(qPreds[i])
        qValuesMemory[{{start_row, end_row}}]:copy(qValues[i])

        if adapt then
            regMemory[{{start_row, end_row}}]:copy(regPreds[i])
        end

        for j = 1, n do
            recall, prec, f1 = rougeScores( Tokenize(totalPredsummary[j]:totable()),
                                            qTokens[j]
                )
            rougue_scores[i][j]:fill(f1)
        end

        if i == n_s then 
            rouguef1[epoch] = rougue_scores[i]:mean()
        end 

        if i == 1 then
            -- Calculating change in rougue f1
            rewards[i]:copy(rougue_scores[i])
        else 
            rewards[i]:copy(rougue_scores[i] - rougue_scores[i-1])
        end
        rewardMemory[{{start_row, end_row}}]:copy(rewards[i])
    end

    if memfull then 
        memrows = memsize
    else 
        memrows = curr_memsize
    end

    if usecuda then 
        dataloader = dl.TensorLoader({
                        queryMemory[{{1, memrows}}]:cuda(), 
                        sentenceMemory[{{1, memrows}}]:cuda(), 
                        predSummaryMemory[{{1, memrows}}]:cuda(),
                        qPredsMemory[{{1, memrows}}]:cuda(), 
                        ByteTensor(memrows, 2):copy(qActionMemory[{{1, memrows}}]), 
                        qValuesMemory[{{1, memrows}}]:cuda(),
                        sentencetp1Memory[{{1, memrows}}]:cuda(),
                        predSummarytp1Memory[{{1, memrows}}]:cuda()               
                            }, 
                    rewardMemory[{{1, memrows}}]:cuda()
                )
        if adapt then            
            table.insert(dataloader['inputs'], regMemory[{{1, memrows}}]:cuda() )
        end
    else 
        dataloader = dl.TensorLoader({
                    queryMemory[{{1, memrows}}], 
                    sentenceMemory[{{1, memrows}}], 
                    predSummaryMemory[{{1, memrows}}], 
                    qPredsMemory[{{1, memrows}}], 
                    ByteTensor(memrows, 2):copy(qActionMemory[{{1, memrows}}]), 
                    qValuesMemory[{{1, memrows}}],
                    sentencetp1Memory[{{1, memrows}}],
                    predSummarytp1Memory[{{1, memrows}}]                    
                    }, 
                rewardMemory[{{1, memrows}}]
            )
        if adapt then
            table.insert(dataloader['inputs'], regMemory[{{1, memrows}}] )
        end
    end
    loss = {}
    c = 1
    for k, xin, reward in dataloader:sampleiter(batch_size, memsize) do
        local function feval(params)
            gradParams:zero()
            if adapt then
                local predtp1 = model:forward({xin[1], xin[7], xin[8]})
                local predQOnActions = maskLayer:forward({xin[4], xin[5]}) 
                local ones = torch.ones(reward:size(1)):resize(reward:size(1))
                if usecuda then
                    ones = ones:cuda()
                end
                lossf = criterion:forward({predQOnActions, xin[7]}, {reward, ones})
                local gradOutput = criterion:backward({predQOnActions, xin[6]}, {reward, ones})
                local gradMaskLayer = maskLayer:backward({xin[4], xin[5]}, gradOutput[1])
                model:backward({xin[1], xin[2], xin[3]}, {gradMaskLayer[1], gradOutput[2]})
            else
                model:forget()
                local predtp1 = model:forward({xin[1], xin[7], xin[8]})
                local predtp1max, _ = torch.max(predtp1, 2)
                model:forget()
                local predt = model:forward({xin[1], xin[2], xin[3]})
                local y_j = reward + (gamma * predtp1max) 
                local predQOnActions = maskLayer:forward({predt, xin[5]}) 
                lossf = criterion:forward(predQOnActions, y_j )
                local gradOutput = criterion:backward(predQOnActions, y_j)
                local gradMaskLayer = maskLayer:backward({predt, xin[5]}, gradOutput)
                model:backward({xin[1], xin[2], xin[3]}, gradMaskLayer[1])

            end 
            return lossf, gradParams
        end
        --- optim.rmsprop returns \theta, f(\theta):= loss function
         _, lossv  = optim.rmsprop(feval, params, optimParams)
        loss[c] = lossv[1]
        c = c + 1
    end

    lossfull[epoch] = Tensor(loss):sum() / #lossv
    if print_perf then
        print(
            string.format('epoch = %i; rougue = %.6f; epsilon = %.6f; loss = %.6f' , 
                epoch, rouguef1[epoch], epsilon, lossfull[epoch])
            )
    end

    if (epsilon - delta) <= base_explore_rate then
        epsilon = base_explore_rate
        if epoch > end_baserate then 
            base_explore_rate = 0.
        end
    else 
        epsilon = epsilon - delta
    end

end
print(string.format("Elapsed time: %.5f" % (os.clock()-nClock) ))
print(
    string.format('First rougue = %.6f; Last rougue = %.6f',
        rouguef1[1], rouguef1[nepochs]) 
    )
